# Installs

In [0]:
%pip install langgraph==0.2.74 databricks-langchain==0.3.0

In [0]:
dbutils.library.restartPython()

# Setup

In [0]:
import requests
from typing import List

from databricks_langchain import ChatDatabricks

# Create A Web Search Tool

In [0]:
from langchain_core.tools import tool

@tool
def execute_bing_web_search(query: str) -> List[str]:
    """
    Runs a Bing web search for a given query.
    Returns a list of snippets of the top 20 results.
    """
    endpoint = "https://api.bing.microsoft.com/v7.0/search"

    params = {
        "q": query, 
        "mkt": "en-GB",
        "safeSearch": "Moderate",
        "count": 20
    }

    headers = {"Ocp-Apim-Subscription-Key": dbutils.secrets.get(<<SCOPE>>, <<KEY>>)}

    #Call the API
    response = requests.get(endpoint, headers=headers, params=params, timeout=10)
    response.raise_for_status()

    found_pages = response.json().get('webPages', {})

    return [v.get("snippet") for v in found_pages["value"]]


In [0]:
print(f"Name: {execute_bing_web_search.name}\n")
print(f"Description: {execute_bing_web_search.description}\n")
print(f"Args: {execute_bing_web_search.args}")

# Create LLM + ReAct Agent

In [0]:
llm = ChatDatabricks(
    endpoint= "gpt-4o"
)

In [0]:
prompt = "You are an AI agent who answers questions. You have access to a web search tool if you need to find out any information. You do not have to use the tools available to you if you are certain of the answer."

In [0]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(llm, [execute_bing_web_search], prompt=prompt)

# Use The Tools!

## A question which the LLM won't know

In [0]:
result = agent_executor.invoke({"messages": [("user", "Who is the Prime Minister of the UK in 2025?")]})

## A question which the LLM will know

In [0]:
result = agent_executor.invoke({"messages": [("user", "What was the capital city of the UK in the year 2004?")]})

## Asking subsequent questions

In [0]:
result = agent_executor.invoke(
    {
        "messages": [
            ("user", "Who is the Prime Minister of the UK in 2025?"),
            ("user", "What about Canada?"),
        ]
    }
)